In [1]:
from data_loader import loadData
from model import RNN
from config import paths, param, hyper
numEpochs = hyper['NumEpoch']
learningRate = hyper['LearningRate']
batchSize = hyper['BatchSize']

sequenceLength = hyper['SlideWindowSize']
hiddenSize = hyper['HiddenLayers']
numLayers = hyper['NumLayers']
# inputSize = param['InputDimension']
outputSize = param['OutputDimension']

datasetPath = paths['Data']
testProportion = param['TestSetProportion']
preloadModelFlag = param['PreloadModelFile']

In [2]:

label, index, ndate, dataX, dataY, trainX, trainY, testX, testY = loadData(datasetPath, sequenceLength, testProportion)

In [3]:
print("dataX: ", dataX.size())
print("dataY: ", dataY.size())
print("trainX: ", trainX.size())
print("trainY: ", trainY.size())
print("testX: ", testX.size())
print("testY: ", testY.size())

dataX:  torch.Size([732, 20, 33])
dataY:  torch.Size([732, 33])
trainX:  torch.Size([659, 20, 33])
trainY:  torch.Size([659, 33])
testX:  torch.Size([73, 20, 33])
testY:  torch.Size([73, 33])


In [11]:
print("trainX: ", trainX[640:trainSize].size())
print("trainY: ", trainY[640:trainSize].size())


trainX:  torch.Size([19, 20, 33])
trainY:  torch.Size([19, 33])


In [7]:
trainSize = len(list(trainX))
print(trainSize)

659


In [3]:
import torch
from torch import nn
# from nni.retiarii.nn.pytorch import nn
import torch.nn.functional as F


In [13]:
class RNN(nn.Module):
    def __init__(self, output_size, input_size, hidden_size, num_layers, sequence_size, batch_size):
        super(RNN, self).__init__()
        self.output_size = output_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.sequence_size = sequence_size
        self.batch_size = batch_size
        
        # self.rnn = nn.LSTM(input_size,hidden_size,num_layers, batch_first=True)
        self.rnn = nn.GRU(input_size,hidden_size,num_layers, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )
    def forward(self, inputX):
        rnn_output, _ = self.rnn(inputX)
        rnn_output = rnn_output.contiguous().view(-1, self.hidden_size)
        output = self.mlp(rnn_output)
        output = output.contiguous().view(self.batch_size, self.sequence_size, self.output_size)
        return output

In [14]:
sequence_size = 5
batch_size = 139
input_size = 1
hidden_size = 2
num_layers = 3
output_size = 1
model = RNN(output_size, input_size, hidden_size, num_layers, sequence_size, batch_size)
inputX = torch.randn(batch_size,sequence_size,input_size)
out = model(inputX)

In [15]:
print(out.size())
print()

torch.Size([139, 5, 1])

